In [3]:
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv("./.env.local")
import os
BEARER_TOKEN = os.getenv("BEARER_TOKEN")
import tweepy
client = tweepy.Client(BEARER_TOKEN, wait_on_rate_limit=True)
from pathlib import Path

In [10]:
# Preprocessing the datasets and save them in csv files under csvdataframes folder
Path.mkdir(Path("csvdataframes"), exist_ok=True)
dataset_path = Path("dataset")

for idx, day in enumerate(dataset_path.iterdir()):
    full_dataset = pd.read_csv(day, compression='gzip')
    df_en = full_dataset[full_dataset['language']=='en']
    df_en_filteted = df_en[["userid", "tweetid", "text", "hashtags"]]
    df_no_duplicate = df_en_filteted.drop_duplicates(subset='text', keep='first')
    df_sampled = df_no_duplicate.sample(15)
    df_sampled.to_csv(Path(f"csvdataframes/day_{idx}.csv"))

C:\Users\Bao\AppData\Local\Temp\ipykernel_18084\2421439641.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')


In [4]:
# Method to call on each csv day files
def create_edgelist(df_sample):  
    for index, row in df_sample.iterrows():
        tweetid = row['tweetid']
#         print(f"{tweetid=}")
        tweet = client.get_tweet(id=tweetid, expansions='referenced_tweets.id')
        
        # Deleted tweets are still in the dataset => just skip them for now
        if (tweet.errors):
            continue
            
        og_tweet_id = tweet.includes.get('tweets')[0].id if tweet.includes.get('tweets') else tweetid
        print(f"{og_tweet_id=}")

        # Get users who retweeted
        users_retweet = client.get_retweeters(id=og_tweet_id)
        
        # Skip if no retweet
        if not users_retweet.data:
            print("No retweet")
            continue
        print(f"{users_retweet.data=}")
            
        for user in users_retweet.data:
            edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
            continue

        next_token = users_retweet.meta.get('next_token')
        print(f"{next_token=}")

        while(next_token):
            users_retweet = client.get_retweeters(id=og_tweet_id, pagination_token=next_token)
            
            if not users_retweet.data:
                print("end of retweet")
                break
            
#             print(f"{users_retweet.data=}")
            
            for user in users_retweet.data:
                edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
                
            next_token = users_retweet.meta.get('next_token')
            print(f"{next_token=}")

        # Get users who liked
        users_like = client.get_liking_users(id=og_tweet_id)
        
        # Skip if no like
        if not users_like.data:
            continue
        
        for user in users_like.data:
            edgelist_like.write(f"{og_tweet_id} {user.id}\n")
            
        next_token = users_like.meta.get('next_token')

        while(next_token):
            users_like = client.get_retweeters(id=og_tweet_id, pagination_token=next_token)
            
            if not users_like.data:
                print("No like")
                break
            
            for user in users_like.data:
                edgelist_like.write(f"{og_tweet_id} {user.id}\n")
                
            next_token = users_like.meta.get('next_token')

In [ ]:
# Main loop
Path.mkdir(Path("edgelists"), exist_ok=True)

#     edgelist_retweet = open(f"edgelists/edgelist_retweet_{time.time()}.txt", "a")
#     edgelist_like = open(f"edgelists/edgelist_like_{time.time()}.txt", "a")

edgelist_retweet = open(f"edgelists/edgelist_retweet.txt", "a")
edgelist_retweet.write("tweet_id rt_user_id\n")

edgelist_like = open(f"edgelists/edgelist_likes.txt", "a")
edgelist_like.write("tweet_id like_user_id\n")
    
for file in Path("csvdataframes").iterdir():
    df = pd.read_csv(file)
    create_edgelist(df)

edgelist_retweet.close()
edgelist_like.close()

Rate limit exceeded. Sleeping for 546 seconds.


og_tweet_id=1498190375495913472
users_retweet.data=[<User id=1495318898916343808 name=Edward_EmpireMonaheng username=empiremonaheng>, <User id=1423666304 name=Nillu username=I_nilmal>]
next_token='7140dibdnow9c7btw480xxgnncg9ts8k60skt5ghrfkfl'
end of retweet
No like
og_tweet_id=1498246909747769344
users_retweet.data=[<User id=1175070207007760385 name=Chrissy Ⓥ Perky Pigoneer 𓃟 username=CHAM02313921>, <User id=1352677660774825984 name=Spencer fletcher 🍃💚🍃 username=Spencerfletch0>, <User id=159095242 name=John Finucane username=johnjoechad>, <User id=1175069750419963906 name=JennyH username=JHam65465689>]
next_token='7140dibdnow9c7btw480xziocns291m4ith1lujjh554i'
end of retweet
No like
og_tweet_id=1498169596884488193
users_retweet.data=[<User id=1472467899390607362 name=Dexter Mavhura username=dexter_mavhura>, <User id=1240669387637633027 name=Francisfxtrader username=francifxtrader1>, <User id=749987670013513730 name=Δ B I K Δ username=realAbikaTrader>, <User id=850075515444703232 name=

users_retweet.data=[<User id=1474042958425079810 name=Whitty’s lost eyebrows (new profile) username=est18wibackup>, <User id=1452627525339803650 name=Futusom username=nkhr_sthan>, <User id=2596625622 name=Kristijan Trupković username=KikiTrupkovic>, <User id=1457417267084464141 name=sığırçobanı username=g_rcoban>, <User id=1351990385216729089 name=Rosesandviolets username=Rose26891183>, <User id=1347654245311516673 name=Drwal username=DrwalX>, <User id=185088197 name=lynnH username=goohugh>, <User id=1210670276574400513 name=Piotrek Kozłowski#SilniRazem🇵🇱🇪🇺 🇺🇦***** *** username=Pete_Kozlowski2>, <User id=1139558657438490624 name=AmirHossein username=amirhossein_sin>, <User id=999738050627416070 name=Saman username=Realsaman26>, <User id=1257964204650192897 name=jaweed username=JennyWeedon>, <User id=3374378669 name=Дон Румата username=DonRoomata>, <User id=420667541 name=Eduardo🌻 username=EduardoBruwurbs>, <User id=1340675325638955011 name=Jothedoggo username=jothedoggo>, <User id=8144

users_retweet.data=[<User id=177143043 name=🌻🌻Jun'ichi Kobayakawa⚖🕊 username=Jun_Kobayakawa>, <User id=549387032 name=afk farm 🇺🇦 username=afk_farm>, <User id=1089192092264480769 name=Legion username=SancheloK>, <User id=1248352761143492609 name=Anabbel Alonso Oficial 🌻🌻🌻 username=aterra2s>, <User id=801726504774365184 name=(((Lior Finkelstein))) ✡️🔴✡ username=Liorfink>, <User id=2893895867 name=Cleiton username=cleiton_j1>, <User id=103538790 name=RiverIbex 🇺🇦 username=RiverIbex>, <User id=2961472313 name=Peglientus #AndreaThinkPositive username=NetworkInquirer>, <User id=1331651021534277636 name=Benfiquista 🦅🇺🇦 username=Suitszz>, <User id=901406062716366848 name=Db Dew username=trunkofbiglove>, <User id=72230361 name=larrise kim⁷ #10robredo #7pangilinan username=larrise_twt>, <User id=1373583724785381384 name=Carlota silva username=Carlota82286703>, <User id=978591673658863616 name=HeadphonesUser25 🇷🇴 username=HeadphonesUsr25>, <User id=980085085000040449 name=Vtrsalvador username=Vt

users_retweet.data=[<User id=24862718 name=Helmer Fonseca username=Helmer_Fonseca>, <User id=22184701 name=Danka Todorova username=angeloglasen>, <User id=2827583311 name=Egle S username=CoolEberry>, <User id=741505382288359424 name=Maria Galan username=MariaGalan83>, <User id=318105991 name=IMEI🍁, Get the jab or lay on a slab username=IMEI21>, <User id=905223920 name=William Jake Henson username=Henson_14>, <User id=1685523818 name=Adrian username=Adrian97a>, <User id=1232799155216666624 name=Chorąży Głuś username=chorazy__glus>, <User id=844723468570705920 name=Sharon username=SharonB53546171>, <User id=931881438940672001 name=VH username=Vvillameh>, <User id=1213863607655518209 name=Aleksandra username=Ola50952721>, <User id=791993624 name=K. Mehrabian username=KMehrabianCCM>, <User id=11162892 name=Lou Sheehan username=LouSheehan>, <User id=910464361285726209 name=Adam Ztych username=AdamZtych>, <User id=93957129 name=mr edinburgh username=mredinburgh>, <User id=261451469 name=Salz

...

users_retweet.data=[<User id=1445109767832281090 name=moi username=Salocin002>, <User id=1493665165488836615 name=Midnight Caller❤🌸🩸 username=FlecksinEffect>, <User id=1357527792 name=FROSH OSHA username=dapo_adeleye01>, <User id=1388644353426460672 name=Hypee username=haha_shampoo>, <User id=1086441338038636544 name=Michael Hernandez username=Michaeljher2883>, <User id=1283505820165177349 name=Artur Mokrzycki username=artur_mokrzycki>, <User id=342999650 name=Sagar Desai username=moveslikesagar>, <User id=459834302 name=A Concerned American username=Ev624>, <User id=884647080886542336 name=Debbie🏳️‍🌈 username=faasdebbiefaas>, <User id=1160249409440870400 name=333 username=47pade>, <User id=1103719363310432258 name=nana username=jinsoulrr>, <User id=1269700494273863680 name=Sola Gratia username=SolaGra04345737>, <User id=3094912196 name=紫電二一型 username=UNCF_BBY_01>, <User id=1494295452157714435 name=SPOOKELSE KLADDEN username=SHABIHA28478500>, <User id=1333072260492476418 name=Paris Cae

users_retweet.data=[<User id=106047063 name=Giò  🧡💙🏛 username=jordiesteban>, <User id=42378816 name=Triinu P. username=trinzu>, <User id=1273308899182026753 name=KPOP2CPOP username=KPOP2CPOP>, <User id=2327922714 name=Jackie Schaeffer username=jaxsch98>, <User id=715712852 name=🇮🇱Сушки к чаю🇺🇦 username=avrmaximus>, <User id=2962598277 name=lol username=DearRules>, <User id=1406361572432359424 name=Alison Hudson username=AlisonH15864665>, <User id=89173305 name=FRANCISCO username=francisco120282>, <User id=1014104789708431360 name=Alex Oxro🇺🇦 🏳️‍🌈 username=AlexOxro>, <User id=1495836293028270080 name=Cikko username=Cikko71472955>, <User id=67923074 name=Mantas username=MrKalenda>, <User id=1197414388656934912 name=vee ♡ 🇺🇦 username=itizvee>, <User id=1102962060911173634 name=Mateusz Czmiel username=MCzmiel>, <User id=1404811136395100163 name=Mikael Tuompo username=MikaelTuompo>, <User id=1260065950260441088 name=Acar Gökhan username=AcarGkhan2>, <User id=276994367 name=Vall. username=V

users_retweet.data=[<User id=3722723175 name=WonderProductions username=wonderprodyt>, <User id=1417890773039730691 name=Arrow 🇵🇱 username=Mateusz43292543>, <User id=716742141788176384 name=Lewis Gildart username=lewis_gildart>, <User id=1214311332092108800 name=Paweł Małecki username=pmal75008066>, <User id=1226224346181488640 name=GMK || ขอไว้อาลัยแก่เหยื่อCOVID-19 🇺🇦🤍 username=kriengkrai_m>, <User id=340716624 name=Safari username=OHTMHF>, <User id=428168856 name=ZłyCzłowiekZLasu username=ZlyCzlowiek>, <User id=938421672297861122 name=Natanieluz username=Natanieluz>, <User id=1496891674676629508 name=Mm username=mm022022>, <User id=1491121939078909952 name=Małgorzata Kubacka username=MKubacka2022>, <User id=1480051489590624256 name=TexasToe username=TexasToe>, <User id=1156271905176989697 name=Barbora Gehlertová username=gehlertova>, <User id=1173369107804303361 name=Gonçalo username=goncalofdias>, <User id=1380055920147886081 name=Ilya username=kravtsev_xxii>, <User id=3558168863 n

No like
og_tweet_id=1497947699018096641
users_retweet.data=[<User id=24734779 name=Rooting for Democracy username=ginatharris>, <User id=1268177129696571397 name=sherri username=sherri26240524>, <User id=512417432 name=Bruce Bourgoine username=BruceBourgoine>]
next_token='7140dibdnow9c7btw480xxgnfpf53xyvxs5yqys78u3hw'
end of retweet
No like
og_tweet_id=1498204425315311619
users_retweet.data=[<User id=1493914178242764803 name=SpazTime username=TurboSpazTime>]
next_token='7140dibdnow9c7btw480xxgnotl4kjg1xq0ezwc1f4rp0'
end of retweet
No like
og_tweet_id=1498292605397999619
No retweet
og_tweet_id=1498147498619244544
users_retweet.data=[<User id=3248745382 name=Víťa Šimek username=n45r4_7>]
next_token='7140dibdnow9c7btw480xxgnncg8s3wbqn7b1i7ax4do0'
end of retweet
No like
og_tweet_id=1497987034182467594
users_retweet.data=[<User id=21765111 name=Reiner Grißhammer username=erlesen>, <User id=573984343 name=Björn Brücher username=BjoernBruecher>, <User id=1248968248042889218 name=Jochen Salber

Rate limit exceeded. Sleeping for 876 seconds.
